### Import packages and API Key

In [128]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
client = OpenAI()
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
from tqdm.auto import tqdm

In [129]:
##pip install elasticsearch==8.11.0 ## used this to ensure using the same version that will run. 

### Load documents in good format

In [130]:
with open('documents.json', 'rt') as f_in: 
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw: 
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

### Create Index Settings for Elasticsearch
#### Note: Looks like persists so trips an error if recreating in same instance? 

In [131]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/_HtjONXjQsmyccDHcb2icg] already exists')

In [132]:
for doc in tqdm(documents): 
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 350.97it/s]


### Create reusable functions. 

In [133]:
def elastic_search(query): 
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [134]:
def build_prompt(query, seach_results): 
    prompt_template = """
    You're a course teaching assistant. Answer the Question based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION. 
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """
    
    context = ""
    
    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
   
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt 

In [135]:
def llm(prompt):
    response=client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )  
    return response.choices[0].message.content

### create RAG 

In [136]:
query  = 'I just discovered the course. Can I still join it?'

In [137]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [138]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."